In [16]:
from transformers import AutoImageProcessor, ResNetModel
import torch
from datasets import load_dataset

In [17]:
from datasets import load_dataset

dataset = load_dataset("frgfm/imagenette", 'full_size')

In [18]:
print(dataset['train'][0]['image'])

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x281 at 0x7F07BA15E2B0>


In [19]:
from transformers import AutoImageProcessor

print('initialize image processor')
image_processor = AutoImageProcessor.from_pretrained("./resnet-50", local_files_only=True)



initialize image processor


In [24]:
image_processor( dataset['train'][0]['image'] )['pixel_values'][0].shape

(3, 224, 224)

In [25]:
dataset['train'][0]['label'] 

2

In [16]:
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.optim as optim
from torch.utils.data import DataLoader, DistributedSampler
from torchvision.models import resnet50
from torchvision.datasets import ImageNet
from torchvision import transforms
from torch.nn.parallel import DistributedDataParallel as DDP

from transformers import AutoImageProcessor, ResNetModel
import torch
from datasets import load_dataset

import os
import io

from torch.utils.data import Dataset
from transformers import AutoImageProcessor
from PIL import Image
import torch

class CustomImageDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.hf_dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        # Get the image and label from the Hugging Face dataset
        item = self.hf_dataset[idx]
        image = item['image']
        # print(image.shape)
        # Check if the image needs to be opened from a bytes-like object
        # if not isinstance(image, Image.Image):
        #     image = Image.open(io.BytesIO(image)).convert("RGB")

        # print(image.shape)
        # Apply the transformation
        # if self.transform:
        image = self.transform(images=image)['pixel_values'][0]

        # Labels can be handled here if needed
        label = item.get('label', torch.tensor(-1))  # Dummy label handling

        return image, label

print('load dataset')
# dataset = load_dataset("imagenet-1k", cache_dir='/home/mheuill/scratch')
test_dataset = dataset['train']  # Select the test split

print('initialize image processor')
image_processor = AutoImageProcessor.from_pretrained("./resnet-50", local_files_only=True)



load dataset
initialize image processor


In [17]:
print('create custom dataset')
custom_dataset = CustomImageDataset(test_dataset, transform=image_processor)


create custom dataset


In [18]:


print('load dataloader')
dataloader = DataLoader(custom_dataset, batch_size=1024,  num_workers=4)

load dataloader


In [3]:
# dataset = load_dataset("imagenet-1k")
# image = dataset["test"]["image"][0]

from transformers import AutoImageProcessor
image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")

image_processor.save_pretrained("./resnet-50")
# inputs = image_processor(image, return_tensors="pt")

# model = ResNetModel.from_pretrained("microsoft/resnet-50")



# with torch.no_grad():
#     outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state
# list(last_hidden_states.shape)

['./resnet-50/preprocessor_config.json']

In [2]:
image_processor

ConvNextImageProcessor {
  "_valid_processor_keys": [
    "images",
    "do_resize",
    "size",
    "crop_pct",
    "resample",
    "do_rescale",
    "rescale_factor",
    "do_normalize",
    "image_mean",
    "image_std",
    "return_tensors",
    "data_format",
    "input_data_format"
  ],
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ConvNextImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}

In [14]:
inputs['pixel_values'].shape

torch.Size([1, 3, 224, 224])